In [2]:
!pip3 install deepctr[gpu]

Looking in indexes: https://pypi.doubanio.com/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     |████████████████████████████████| 129 kB 1.6 MB/s eta 0:00:01
  Using cached https://pypi.doubanio.com/packages/b8/f3/9c15ce0b689076ac937b4d6c1d9ee8beea7b0a2f7aeac8aaba1377be3845/tensorflow_gpu-2.5.0-cp36-cp36m-manylinux2010_x86_64.whl (454.3 MB)
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 6.0 MB 55.3 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 56.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 56.0 MB/s eta 0:00:01
  Using cached https://pypi.doubanio.com/packages/ab/f5/3d3bcb82beae990021cbf6877456a1aab650e68c902194566edd6a73e37c/grpcio-1.34.1-cp36-cp36m-manylinux2014_x86_64.whl (4.0 MB)
     |████████████████████████████████| 462 kB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.1 MB/s  eta 0:00:01
     |███████████████

In [24]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score,f1_score
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr.models import DeepFM,xDeepFM ,FGCNN, NFM#NFFM,
#from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr.feature_column import SparseFeat,DenseFeat,get_feature_names
from gensim.models import Word2Vec
from tensorflow.keras.callbacks import *
import tensorflow as tf
from tensorflow.keras.optimizers import *

In [2]:
train = pd.read_csv('./data/new_data/train_data.txt',sep='\t')
test = pd.read_csv('./data/new_data/test_data.txt',sep='\t')
#data = pd.concat([train, test])
train["launch_seq"] = train.launch_seq.apply(lambda x: json.loads(x))
train["playtime_seq"] = train.playtime_seq.apply(lambda x: json.loads(x))
train["duration_prefer"] = train.duration_prefer.apply(lambda x: json.loads(x))
train["interact_prefer"] = train.interact_prefer.apply(lambda x: json.loads(x))

In [3]:
data['tagid'] = data['tagid'].apply(lambda x: eval(x))

In [4]:

sentences = data['tagid'].values.tolist()
for i in range(len(sentences)):
    sentences[i] = [str(x) for x in sentences[i]]   #将每个tagid转化成str格式

emb_size = 128
#model = Word2Vec(sentences,vector_size=emb_size, window=10, min_count=1, sg=0, hs=0, seed=1,epochs=5)
model = Word2Vec.load('./w2vmodel/w2vmodel.model')
emb_matrix = []
for seq in sentences:
    vec = []
    for w in seq:
#         if w in model.wv.vocab:
#             vec.append(model.wv[w])
        try:
            vec.append(model.wv[w])
        except KeyError:
                continue
                
    if len(vec) > 0:
        emb_matrix.append(np.mean(vec, axis=0))
    else:
        emb_matrix.append([0] * emb_size)
emb_matrix = np.array(emb_matrix)
for i in range(emb_size):
    data['tag_emb_{}'.format(i)] = emb_matrix[:, i]

In [5]:
#增加model特征
def replace_dup(data):
    data = data.replace('vivo',"VIVO")
    data = data.replace('Moto',"摩托罗拉")
    data = data.replace('索尼移动',"索尼")
    data = data.replace('移动',"中国移动")
    data = data.replace('8848钛金手机',"8848")
    
    return data
data['make'] = replace_dup(data['make'])

vs = ['华为', '魅族', 'OPPO', '360', '荣耀', '小米', '联想', '努比亚', 'VIVO',
        '金立', '邦华', '锤子科技', '乐视', '海信', '一加', '苹果', '美图', '中国移动',
          '中兴', '魅果', '酷比', '小辣椒', '国美', '小天才', '酷派',
      '优博讯', '8848', '虹动', '康佳', '索尼', 'COMIO', '朵唯', '进化者',
         '夏普', '长虹', '格力', 'ALLWINNER', '创维', '黑莓', '好爱', '诺亚信',
        '天语', '三星', '贝尔丰', 'SUGAR', '极豆DVD导航', '读书郎',
         '众赢时代', 'AGM', 'ivvi', '赛博宇华', '米语',
        '东方拓宇',  'alps', '欧亚信', 'vetas', '传奇']
vs1 = ['华为', '魅族', 'OPPO', '360', '荣耀', '小米', '联想', '努比亚', 'VIVO',
        '金立', '邦华', '锤子科技', '乐视', '海信', '一加', '苹果', '美图', '中国移动',
          '中兴','酷比', '小辣椒', '小天才', '酷派']
data['make'] = data['make'].apply(lambda x:x if x in vs1 else "未知")

In [6]:
data['tagidLen'] = data['tagid'].apply(lambda x:len(x))
data['age'] = data['age'].fillna(0)
data['gender'] = data['gender'].fillna(2)

In [7]:
data.columns

Index(['pid', 'label', 'gender', 'age', 'tagid', 'time', 'province', 'city',
       'make', 'model',
       ...
       'tag_emb_119', 'tag_emb_120', 'tag_emb_121', 'tag_emb_122',
       'tag_emb_123', 'tag_emb_124', 'tag_emb_125', 'tag_emb_126',
       'tag_emb_127', 'tagidLen'],
      dtype='object', length=139)

In [8]:
features = [i for i in data.columns if i not in ['pid', 'tagid', 'time', 'model','city']]  ##尝试加入用户pid，过拟合
data = data[features]

In [9]:
sparse_features = ['province', 'make']
dense_features = ['gender', 'age','tagidLen']
emb_feat = ['tag_emb_{}'.format(i) for i in range(emb_size)]
# for i in data.columns:
#     if data[i].nunique() > 1000:
#         dense_features.append(i)
#     else:
#         if i == "label":
#             break
#         sparse_features.append(i)

# data[sparse_features] = data[sparse_features].fillna('-1', )
# data[dense_features] = data[dense_features].fillna(0, )
# target = ['label']

# # 1.Label Encoding for sparse features,and do simple Transformation for dense features
#fm层的类别数据要先编码，然后会有个embedding操作
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

#dense的连续数据标准化
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])
#print(data.head())
# # 2.count #unique features for each sparse field,and record dense feature field name

#深层特征列
#将稀疏特征进一步通过嵌入技术将其转成稠密向量，将稠密特征拼接到全连接神经网络的输入向量
#标签编码
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim="auto")
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features] 
#哈希编码
# fixlen_feature_columns = [SparseFeat(feat,vocabulary_size=data[feat].nunique(),embedding_dim=4,use_hash=True)
#                         for feat in sparse_features] + [DenseFeat(feat, 1, )
#                       for feat in dense_features]

#加入embeding特征
fixlen_feature_columns.extend([DenseFeat(i, 1) for i in emb_feat])

#生成特征列
dnn_feature_columns = fixlen_feature_columns  #用做DNN模型的输入向量
linear_feature_columns = fixlen_feature_columns  #用做线性模型的输入特征

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)#获取所有特征名

#all_feature = feature_names + emb_feat

In [10]:
data[feature_names].head(-1)

,province,make,gender,age,tagidLen,tag_emb_0,tag_emb_1,tag_emb_2,tag_emb_3,tag_emb_4,...,tag_emb_118,tag_emb_119,tag_emb_120,tag_emb_121,tag_emb_122,tag_emb_123,tag_emb_124,tag_emb_125,tag_emb_126,tag_emb_127
0,13,8,1.0,0.000000,0.079127,-0.584356,-0.505823,0.163237,-0.118881,-0.570097,...,-0.199300,-0.039976,0.120165,0.704987,0.455543,-0.556336,0.340716,-0.544948,-0.441241,-0.371886
1,12,1,1.0,0.833333,0.083220,-1.238055,-0.382685,-0.172527,-1.336101,-0.141902,...,-0.149191,0.512553,-0.193722,0.661562,-0.182866,-0.928568,-0.151268,0.027404,0.356565,0.194155
2,2,10,1.0,0.000000,0.065484,-0.207355,-0.698800,-0.361576,-0.397024,-0.512204,...,-0.337251,-1.367355,-0.110100,0.931193,-0.368334,-0.824169,-0.105975,-0.491890,-0.327030,-0.480906
3,6,2,1.0,0.333333,0.095498,-0.779528,-0.823690,-0.159587,0.393944,-0.301837,...,0.374327,0.085621,-0.115341,0.710098,0.498155,-0.452055,0.672465,-0.756169,-0.096882,-0.567550
4,22,2,1.0,0.000000,0.090041,-0.811674,-0.868393,0.203260,0.186939,-0.333352,...,0.087187,-0.127469,0.075751,0.609228,0.279520,-0.011757,0.636445,-0.375453,-0.311878,-0.438952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,16,2,1.0,0.000000,0.042292,-1.075894,-0.557236,0.592413,0.416230,-0.704641,...,0.416929,-0.094732,-0.341141,0.928138,0.412233,-0.262490,0.705931,-0.806319,-0.183453,-0.060224
99995,10,1,1.0,0.166667,0.058663,-1.200765,-0.281757,-0.261214,-0.896719,-0.183152,...,-0.274514,0.602111,-0.016358,0.659440,0.128409,-0.733430,-0.100913,-0.011235,0.463033,0.383730
99996,9,1,1.0,0.500000,0.040928,-0.974629,0.105890,0.334419,-0.775297,-0.085035,...,-0.455192,0.718217,-0.123305,0.863213,0.032837,-0.589624,0.027972,-0.011204,0.159558,0.297565
99997,10,2,1.0,0.000000,0.006821,-0.513226,-0.143244,0.693421,0.869403,-0.162662,...,0.252258,-0.838978,-1.282002,1.021543,0.761146,-0.754921,0.750287,-1.533581,0.073937,0.437128


In [27]:
# # 3.generate input data for model
newtrain = data[data['label']!= 2]
test = data[data['label']==2]
test_model_input = {name: test[name] for name in feature_names}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2048)
oof = np.zeros([len(newtrain), 1])
predictions = np.zeros([len(test), 1])

for i, (trn_idx, val_idx) in enumerate(folds.split(newtrain, newtrain['label'])):
    print("fold n{}".format(i + 1))
    tr_x = newtrain.iloc[trn_idx,:]
    tr_y = newtrain.iloc[trn_idx]['label']
    val_x = newtrain.iloc[val_idx,:]
    val_y = newtrain.iloc[val_idx]['label']
    train_model_input = {name: tr_x[name] for name in feature_names}  #每个特征都是一个input
    val_model_input = {name: val_x[name] for name in feature_names}

    # 4.Define Model,train,predict and evaluate
    # model = NFFM(linear_feature_columns, dnn_feature_columns, l2_reg_dnn=0.01, dnn_dropout=0.5, task='binary')
    #model = FGCNN(dnn_feature_columns,l2_reg_dnn=0.01,dnn_dropout=0.5,task='binary')
    model = DeepFM(linear_feature_columns, dnn_feature_columns,dnn_hidden_units=[256,256,128], l2_reg_dnn=0.001, dnn_dropout=0.25, task='binary')

    #if fold_ == 0:
    #model.summary()

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=Adam(lr=0.001),metrics=['accuracy'], )
   
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, mode='auto',min_delta=0.0001, verbose=2)  #改监控指标为val_acc
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=5,verbose=2)
    bst_model_path = "./model/deepfm_{}.h5".format(1)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

    history = model.fit(train_model_input, tr_y,
                    batch_size=256, epochs=10000, verbose=2, validation_data=(val_model_input,val_y),callbacks=[early_stopping, model_checkpoint,reduce_lr])

    model.load_weights(bst_model_path)    
    oof[val_idx] = model.predict(val_model_input, batch_size=256)
    predictions += model.predict(test_model_input, batch_size=256)
print("val LogLoss", round(log_loss(newtrain['label'].values, oof), 4))
print("val AUC", round(roc_auc_score(newtrain['label'].values, oof), 4))   

fold n1


/root/.local/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/10000
938/938 - 15s - loss: 0.7003 - accuracy: 0.6793 - val_loss: 0.6154 - val_accuracy: 0.6793
Epoch 2/10000
938/938 - 11s - loss: 0.5966 - accuracy: 0.6908 - val_loss: 0.6020 - val_accuracy: 0.6801
Epoch 3/10000
938/938 - 11s - loss: 0.5910 - accuracy: 0.6926 - val_loss: 0.5951 - val_accuracy: 0.6844
Epoch 4/10000
938/938 - 11s - loss: 0.5885 - accuracy: 0.6930 - val_loss: 0.5904 - val_accuracy: 0.6902
Epoch 5/10000
938/938 - 11s - loss: 0.5861 - accuracy: 0.6950 - val_loss: 0.5884 - val_accuracy: 0.6924
Epoch 6/10000
938/938 - 11s - loss: 0.5851 - accuracy: 0.6957 - val_loss: 0.5851 - val_accuracy: 0.6947
Epoch 7/10000
938/938 - 11s - loss: 0.5841 - accuracy: 0.6958 - val_loss: 0.5886 - val_accuracy: 0.6888
Epoch 8/10000
938/938 - 11s - loss: 0.5843 - accuracy: 0.6947 - val_loss: 0.5896 - val_accuracy: 0.6905
Epoch 9/10000
938/938 - 11s - loss: 0.5832 - accuracy: 0.6966 - val_loss: 0.5876 - val_accuracy: 0.6910

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0001

ValueError: Found input variables with inconsistent numbers of samples: [300000, 60000]

In [29]:
print("val LogLoss", round(log_loss(newtrain['label'].values, oof), 4))
print("val AUC", round(roc_auc_score(newtrain['label'].values, oof), 4))   

val LogLoss 0.5654
val AUC 0.7771


In [27]:
from tensorflow.keras.utils import plot_model
plot_model(model,to_file="model.png",show_shapes=True)


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [31]:
submit = oritest[['pid']]
submit['proba'] = predictions
submit.columns = ['user_id', 'proba']

submit = submit.sort_values(['proba'],ascending=False)
submit = submit.reset_index(drop=True)
submit['category_id'] = 1
submit.loc[int(len(submit)*0.5):,'category_id'] = 0

submit[['user_id', 'proba']].to_csv('./sub/sub_proba_deepfm.csv', index=False)
submit[['user_id', 'category_id']].to_csv('./sub/sub_deepfm.csv', index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
